### Naive Approaches

In [1]:
from operator import mul
from functools import reduce

In [2]:
def naive_factorial(n):
    res = 1
    for i in range(2, n+1):
        res *= i
    return res

In [3]:
def naive_factorial_2(n):
    return reduce(mul, list(range(1, n+1)))

### Dynamic Programming

In [4]:
def factorial_table(N):
    # the most sensible way to compute a sequence of factorials
    # is to use the previous terms
    table = [1] * N
    for i in range(1, N):
        table[i] = table[i-1] * (i+1)
    return table

factorial_table(6)

[1, 2, 6, 24, 120, 720]

### Math

In [5]:
import math
import numpy as np
from scipy.special import gamma

In [6]:
def stirling(n):
    return math.sqrt(2*math.pi*n)*pow((n/math.e), n)

In [7]:
def gamma_factorial(n):
    # note this is not exact
    # gamma_factorial(100) != math.factorial(100)
    return gamma(n + 1)

### Tree Based Factorial Algorithms

In [8]:
import multiprocessing as mp

from operator import mul
from functools import reduce
from itertools import starmap

In [9]:
def binary_factorial(n):
    # break up n factorial into n smaller prods
    # (1, 2), (2, 3), ... (n-1, n)
    if n % 2 == 0:
        values = starmap(mul, zip(range(1, n+1, 2), range(2, n+1, 2)))
        return reduce(mul, values)
    else:
        values = starmap(mul, zip(range(1, n+1, 2), range(2, n+1, 2)))
        return reduce(mul, values) * n

In [10]:
def mp_binary_factorial(n, NUM_CORES=16):
    # same methodology as binary factorial
    with mp.Pool(processes=NUM_CORES) as pool:
        values = pool.starmap(mul, zip(range(1, n+1, 2), range(2, n+1, 2)))
    
    if n % 2 != 0:
        values += [n]
    
    return reduce(mul, values)

In [11]:
def binary_factorial_2(n):
    # break up the factorial into n smaller prods
    # but divide the entire second sequence by 2
    # pulling the term out in front
    if n % 2 == 0:
        values = starmap(mul, zip(range(1, n+1, 2), range(1, n+1, 1)))
        return reduce(mul, values) * ( 1<< (n//2) )
    else:
        values = starmap(mul, zip(range(1, n+1, 2), range(1, n//2 + 1, 1)))
        return reduce(mul, values) * ( 1<< (n//2) ) * n

### Applications of a Prime Factorization
Consider $6!$
$$6! = 1 \cdot 2 \cdot 3 \cdot 4 \cdot 5 \cdot 6 = 2^4 \cdot 3^2 \cdot 5 = 720$$

We can see that in the sequence $1, 2, 3, 4, 5, 6$ 
- 2 occurs 4 times
- 3 occurs twice
- 5 appears once

Is there a closed form function to compute the numbers of times each prime appears between 1 and n?
*yes*, is it called [Legendre's Formula](https://en.wikipedia.org/wiki/Legendre%27s_formula)

**Legendre's Formula**

Let $\nu_p(n!)$ denote the number of times $p$ occurs in $n!$

$$\nu_p(n!) = \sum_{i}^n \frac{n}{p^i}$$

This is actually trivial to prove. 
- Start with a prime, eg. 2, it has 3 multiples less than or equal to 6 (6/2 = 3)
- Note that we only counted 4 once, despite the fact that it holds 2 2's. So add the additional 2 (6/4 = 1)

In [12]:
def legendre(n, p):
    return sum([n//p**i for i in range(1, n)])

legendre(6, 2), legendre(6, 3), legendre(6, 5)

(4, 2, 1)

Finding all of the prime factors of an integer $n$ using the sieve of eratosthenes

In [13]:
import numpy as np
import math

In [14]:
def eratosthenes_sieve_v3(N, include_one=False):
    l = np.ones(N, dtype=np.bool)
    
    find_multiples = lambda n, C : [i*n-1 for i in range(2, C//n +1)]
    upper_bound = math.ceil(math.sqrt(N))
    for val in range(2, upper_bound+1):
        if l[val-1]:
            multiples = find_multiples(val, N)
            l[multiples] = False
            
    primes = set((np.nonzero(l)[0] + 1).tolist())
    if not include_one:
        primes.remove(1)
    return primes

In [15]:
def prime_factorial(n):
    res = 1
    for p in eratosthenes_sieve_v3(n):
        alpha = legendre(n, p)
        if p == 2:
            res <<= alpha
        else:
            res *= pow(p, alpha)
            
    return res

### Moessnar and the Triangular Factorials
[Link to Blog Post](https://topic.alibabacloud.com/a/another-interesting-algorithm-for-calculating-font-colorredfactorialfont_8_8_31867939.html)

### Python Internals
The algorithm used to multiply smaller integers is not the same as the algorithm used to multiply larger integers. You can see [here](https://github.com/python/cpython/blob/main/Objects/longobject.c#L81) in the python internals, that if an integer is more than 70 digits long, the Karatsuba multiplication algorithm is used.